In [1]:
import numpy as np 
import pandas as pd 

import matplotlib.pyplot as plt
#import plotly.express as px
import seaborn as sns

from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import StratifiedKFold
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, \
                            accuracy_score, f1_score, average_precision_score, \
                            fbeta_score, precision_recall_curve, auc
from sklearn.metrics import matthews_corrcoef

from sklearn.model_selection import train_test_split

import optuna
from optuna.samplers import TPESampler
sns.set(style='white', context='notebook', palette='deep')
%config Completer.use_jedi = False

In [2]:
csl_df = pd.read_csv('../../data/csl/Sites/CSL_he_PI_s50.csv', index_col=0)
X = csl_df.drop(['high_EBL','Sitenum'], axis=1, inplace=False)
y = csl_df['high_EBL'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=7)
X_train                                                    

,transfus_yes,Accrete,AdmBishop,Admcervpos,Admconsistency,Admcontract,AdmDBP,Admefface,Admpresent,AdmSBP,...,TD_nos,ThreatenedPB,threatpb9,UnspecHBP,uscar,version9,new_age,new_BMI,new_high_Age,new_high_BMI
MomID,,,,,,,,,,,,,,,,,,,,,
50-13687,0,0,8,2,3,2.0,75,70.0,1,123.0,...,0,0,0,0,0,0,24,24,1,1
50-08549,0,0,9,2,3,3.0,77,80.0,1,128.0,...,0,0,0,0,0,0,35,24,1,1
50-14151,0,0,13,1,3,2.0,78,90.0,1,124.0,...,0,0,0,0,0,0,22,24,1,1
50-00959,0,0,8,8,8,2.0,60,0.0,77,107.0,...,0,0,0,0,1,0,41,24,2,1
50-15186,0,0,8,8,8,2.0,77,0.0,77,124.0,...,0,0,0,0,0,0,34,24,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50-10286,0,0,8,2,3,1.0,73,80.0,1,104.0,...,0,0,0,0,0,0,22,20,1,1
50-10663,0,0,8,8,8,3.0,62,0.0,77,117.0,...,0,0,0,0,0,0,29,24,1,1
50-07713,0,0,8,8,8,3.0,72,90.0,1,125.0,...,0,0,0,0,0,0,29,24,1,1


In [3]:
X.shape

(11175, 195)

<h1 align='center'>Exploratory Data Analysis</h1>

<h1 align='center'>Modelling</h1>

In [8]:
print(X_train.shape)
print(X.shape)
print(y.shape)

(7822, 195)
(11175, 195)
(11175,)


Here we define our validation set

<h1 align='center'>Modelling Using Undersampling</h1>

We will use `imblearn`'s RandomUnderSampler to undersample from the majority class so that they match

In [9]:
sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
X_rus, y_rus = sampler.fit_resample(X_train, y_train)

# Try with no undersampling
#X_rus, y_rus = X_train, y_train

In [10]:
print(X_rus.shape)
print(y_rus.shape)
print(np.bincount(y_rus))

(938, 195)
(938,)
[469 469]


<h1 align='center'>Basic Gradient Boosting</h1>

In [11]:
%%time
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_rus = GradientBoostingClassifier()   # If not undersampling
gb_rus.fit(X_rus, y_rus)
print(gb_rus.get_params())
y_pred = gb_rus.predict(X_test)
probs = gb_rus.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[2468  684]
 [  24  177]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.87      3152
           1       0.21      0.88      0.33       201

    accuracy                           0.79      3353
   macro avg       0.60      0.83      0.60      3353
weighted avg       0.94      0.79      0.84      3353

ROC AUC prob1: 0.894348845872162
MCC: 0.36059245889417924
F2: 0.5315315315315315
PR_AUC: 0.34201454182809427
CPU times: user 389 ms, sys: 2.86 ms, total

In [12]:
%%time
# Not undersampled
from sklearn.metrics import matthews_corrcoef
from sklearn.utils import class_weight
gb_weight = GradientBoostingClassifier()  
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
print(gb_weight.get_params())
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

{'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'deviance', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'presort': 'deprecated', 'random_state': None, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
[[2516  636]
 [  26  175]]
              precision    recall  f1-score   support

           0       0.99      0.80      0.88      3152
           1       0.22      0.87      0.35       201

    accuracy                           0.80      3353
   macro avg       0.60      0.83      0.61      3353
weighted avg       0.94      0.80      0.85      3353

ROC AUC prob1: 0.8926449604768038
MCC: 0.37079573149315526
F2: 0.5417956656346749
PR_AUC: 0.35054167764247335
CPU times: user 2.42 s, sys: 5.2 ms, total

<h1 align='center'>Hyperparameter Tuning with Cross Validation</h1>

In [13]:
%%time
def create_model(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    subsample = trial.suggest_float('subsample', 0.8, 1.0)
    max_depth = trial.suggest_int('max_depth',3,20)
    max_features = trial.suggest_int('max_features',10,20)
    learning_rate = trial.suggest_uniform('learning_rate',0.08,0.2)
    min_samples_leaf = trial.suggest_int('min_samples_leaf',1,50)
    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
                           max_depth=max_depth, learning_rate=learning_rate, 
                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, subsample=subsample, 
#                           learning_rate=learning_rate, 
#                           min_samples_leaf=min_samples_leaf)
#    model = GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate) 
    return model

def objective(trial):
    model = create_model(trial)
    
    mcc_list = []
    roc_list = []
    pr_list = []
    f2_list = []
    cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=11)
    for idx, (train_idx, test_idx) in enumerate(cv.split(X, y)):
        X_train_c, X_test_c = X.iloc[train_idx], X.iloc[test_idx]
        y_train_c, y_test_c = y[train_idx], y[test_idx]
        sampler = RandomUnderSampler(sampling_strategy=1.0, random_state=7)
        X_res, y_res = sampler.fit_resample(X_train_c, y_train_c)

#        weights = class_weight.compute_sample_weight('balanced', y=y_train)
#        model.fit(X_train, y_train, sample_weight=weights)
        model.fit(X_res, y_res)
        y_pred = model.predict(X_test_c)
        probs = model.predict_proba(X_test_c)
        prob1 = probs[:, 1]  # Only positives
        precision, recall, pr_thresh = precision_recall_curve(y_test_c, prob1)
        roc = roc_auc_score(y_test_c, prob1)
        roc_list.append(roc)
        mcc = matthews_corrcoef(y_test_c, y_pred)  # MWB
        mcc_list.append(mcc)
        f2 = fbeta_score(y_test_c, y_pred, beta=2.0, average=None)[1]
        pr_auc = auc(recall, precision)
        pr_list.append(pr_auc)
        avg_precision = average_precision_score(y_test_c, prob1)
#        print(f'ROC: {roc}')
#        print(f'PR_AUC: {pr_auc}')
#        print(f'avg_prec: {avg_precision}')
        
    print(f'MCC: {mcc_list}')
    print(f'ROC: {roc_list}')
    print(f'PR_AUC: {pr_list}')
    return np.mean(mcc_list)

sampler = TPESampler(seed=7)
study = optuna.create_study(sampler=sampler,direction='maximize')
#study.optimize(objective,n_trials=60)
study.optimize(objective,n_trials=25)
#study.optimize(objective,n_trials=5)

[I 2021-12-10 14:35:37,116] A new study created in memory with name: no-name-b7b22ce5-207e-4a74-8088-0f0ca8377269
[I 2021-12-10 14:35:38,122] Trial 0 finished with value: 0.38311164347611826 and parameters: {'n_estimators': 24, 'subsample': 0.955983758448023, 'max_depth': 10, 'max_features': 17, 'learning_rate': 0.1973587414395923, 'min_samples_leaf': 27}. Best is trial 0 with value: 0.38311164347611826.


MCC: [0.37716894682933133, 0.3876087228162542, 0.3889104192862239, 0.37875848497266384]
ROC: [0.8980303572527575, 0.8878069125593878, 0.9086379791825338, 0.8911164723105198]
PR_AUC: [0.32098475408182586, 0.26752594545025743, 0.30960392965705963, 0.3038204947368855]


[I 2021-12-10 14:35:40,652] Trial 1 finished with value: 0.38062334119182795 and parameters: {'n_estimators': 105, 'subsample': 0.8144102266719524, 'max_depth': 7, 'max_features': 15, 'learning_rate': 0.16150759953451288, 'min_samples_leaf': 41}. Best is trial 0 with value: 0.38311164347611826.


MCC: [0.3716694202750342, 0.3757038838803931, 0.3872849104928098, 0.38783515011907455]
ROC: [0.8900501243421036, 0.8848012548507598, 0.8998386102346498, 0.8867816537526623]
PR_AUC: [0.2878372365326239, 0.26390388399314535, 0.2873027030673795, 0.29200991819820094]


[I 2021-12-10 14:35:42,842] Trial 2 finished with value: 0.3858550059689128 and parameters: {'n_estimators': 82, 'subsample': 0.813187269381181, 'max_depth': 8, 'max_features': 20, 'learning_rate': 0.10560624242958987, 'min_samples_leaf': 23}. Best is trial 2 with value: 0.3858550059689128.


...

MCC: [0.37730886336295505, 0.3823093654222866, 0.381465634429466, 0.3933796841718578]
ROC: [0.9008910234346685, 0.8973678961302725, 0.9005412903927756, 0.8942655435511309]
PR_AUC: [0.3245296734698713, 0.2942314830337145, 0.29419100704059975, 0.30060887826731203]


[I 2021-12-10 14:36:48,289] Trial 23 finished with value: 0.3834886998035195 and parameters: {'n_estimators': 69, 'subsample': 0.977104756139798, 'max_depth': 20, 'max_features': 18, 'learning_rate': 0.11496449307660377, 'min_samples_leaf': 32}. Best is trial 21 with value: 0.3890382077666189.


MCC: [0.3645929049890338, 0.3856813117574368, 0.3952217485144742, 0.38845883395313313]
ROC: [0.8941485130234392, 0.8863290175171363, 0.8992265984840243, 0.8920947138472485]
PR_AUC: [0.3176102940578714, 0.2698962460027142, 0.274441838160545, 0.3064146598172]


[I 2021-12-10 14:36:49,455] Trial 24 finished with value: 0.3852080294502233 and parameters: {'n_estimators': 46, 'subsample': 0.9243927389568112, 'max_depth': 6, 'max_features': 19, 'learning_rate': 0.14670437395550778, 'min_samples_leaf': 43}. Best is trial 21 with value: 0.3890382077666189.


MCC: [0.37863465687650566, 0.3857287862856787, 0.3865311671369369, 0.3899375075017719]
ROC: [0.9021401430105149, 0.890848837631016, 0.9021891886990897, 0.8908268763311153]
PR_AUC: [0.3031070265324341, 0.2820028581279868, 0.2819837892098025, 0.2793317795314956]
CPU times: user 1min 12s, sys: 279 ms, total: 1min 12s
Wall time: 1min 12s


In [14]:
best = study.best_trial
best

FrozenTrial(number=21, values=[0.3890382077666189], datetime_start=datetime.datetime(2021, 12, 10, 14, 36, 43, 781671), datetime_complete=datetime.datetime(2021, 12, 10, 14, 36, 44, 866571), params={'n_estimators': 61, 'subsample': 0.9247900894485763, 'max_depth': 3, 'max_features': 19, 'learning_rate': 0.13522323478552553, 'min_samples_leaf': 35}, distributions={'n_estimators': IntUniformDistribution(high=200, low=10, step=1), 'subsample': UniformDistribution(high=1.0, low=0.8), 'max_depth': IntUniformDistribution(high=20, low=3, step=1), 'max_features': IntUniformDistribution(high=20, low=10, step=1), 'learning_rate': UniformDistribution(high=0.2, low=0.08), 'min_samples_leaf': IntUniformDistribution(high=50, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=21, state=TrialState.COMPLETE, value=None)

In [15]:
%%time
gb_params = study.best_params
#gb_params['random_state'] = 42
gb = GradientBoostingClassifier(**gb_params)
gb.fit(X_rus, y_rus)
y_pred = gb.predict(X_test)
probs = gb.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[2451  701]
 [  18  183]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.87      3152
           1       0.21      0.91      0.34       201

    accuracy                           0.79      3353
   macro avg       0.60      0.84      0.60      3353
weighted avg       0.95      0.79      0.84      3353

ROC AUC prob1: 0.8941586483824533
MCC: 0.3707012094152902
F2: 0.5420616113744074
PR_AUC: 0.3468718489818416
CPU times: user 108 ms, sys: 0 ns, total: 108 ms
Wall time: 106 ms


In [16]:
%%time
gb_params = study.best_params
gb_params['random_state'] = 42
gb_weight = GradientBoostingClassifier(**gb_params)
weights = class_weight.compute_sample_weight('balanced', y=y_train)
gb_weight.fit(X_train, y_train, sample_weight=weights)
y_pred = gb_weight.predict(X_test)
probs = gb_weight.predict_proba(X_test)
prob1 = probs[:, 1]  # Only positives
precision, recall, pr_thresh = precision_recall_curve(y_test, prob1)
print(confusion_matrix(y_pred=y_pred, y_true=y_test))
print(classification_report(y_test, y_pred))
print(f'ROC AUC prob1: {roc_auc_score(y_test, prob1)}')
print(f'MCC: {matthews_corrcoef(y_test, y_pred)}')
print(f'F2: {fbeta_score(y_test, y_pred, beta=2.0, average=None)[1]}')
print(f'PR_AUC: {auc(recall, precision)}')

[[2454  698]
 [  19  182]]
              precision    recall  f1-score   support

           0       0.99      0.78      0.87      3152
           1       0.21      0.91      0.34       201

    accuracy                           0.79      3353
   macro avg       0.60      0.84      0.60      3353
weighted avg       0.95      0.79      0.84      3353

ROC AUC prob1: 0.8939061039977775
MCC: 0.3690713284913456
F2: 0.5403800475059383
PR_AUC: 0.33108666410017273
CPU times: user 318 ms, sys: 2.01 ms, total: 320 ms
Wall time: 318 ms


In [17]:
# No Sitenum - 5
coeffs_wgt = pd.Series(data=gb_weight.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs_wgt[0:20]

Delmode            0.217712
Delfetalpos        0.120239
Lac_Min            0.106153
Hxanemia           0.076912
Episiotomy         0.073830
Hxcsection         0.050294
Antefetdistress    0.040191
CS_NRFHT           0.035949
CS_FTP             0.033980
ROMmeth            0.026279
uscar              0.023722
CS_UScar           0.021648
new_age            0.018952
transfus_yes       0.015455
Admefface          0.013004
new_BMI            0.012977
CS_Breech          0.009977
TrialLabor         0.009367
prelaborCD         0.008360
Admcervpos         0.008231
dtype: float64

In [1]:
plt.rcParams["figure.figsize"] = (14, 8)
x_labels = coeffs_wgt.index.values[0:25]
print(x_labels)
fig, ax = plt.subplots(1,1)  # Create a figure and an axes.
ax.set_title('Top Predictors: All Sites combined; target: High_EBLoss, vars: PI')
ax.bar(x_labels, coeffs_wgt.values[0:25])
ax.set_ylabel('Coeff')
ax.set_xlabel('Variable')
plt.draw()
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=16)
#plt.xlim(1,20)
plt.show()

NameError: name 'plt' is not defined

In [19]:
# No Sitenum - 5
coeffs = pd.Series(data=gb.feature_importances_,
                   index=X_test.columns.values).sort_values(ascending=False)
coeffs[0:20]

Delmode            0.425730
Hxanemia           0.110664
Delfetalpos        0.051138
Lac_Min            0.036525
Lac_None           0.036066
CS_NRFHT           0.033599
Admpresent         0.029036
CS_UScar           0.027319
Admefface          0.023339
prelaborCD         0.020943
CS_FTP             0.018013
TrialLabor         0.017209
new_BMI            0.014104
AdmDBP             0.013791
Dilat_lst          0.013083
Episiotomy         0.012295
BESTGA             0.011769
Antefetdistress    0.010323
transfus_yes       0.009052
Hxcsection         0.007714
dtype: float64

In [21]:
gb_params # 5 runs

{'n_estimators': 61,
 'subsample': 0.9247900894485763,
 'max_depth': 3,
 'max_features': 19,
 'learning_rate': 0.13522323478552553,
 'min_samples_leaf': 35,
 'random_state': 42}

In [2]:
from optuna.visualization import plot_optimization_history

plotly_config = {"staticPlot": True}

fig = plot_optimization_history(study)
fig.show(config=plotly_config)

NameError: name 'study' is not defined

In [ ]:
from optuna.visualization import plot_param_importances

fig = plot_param_importances(study)
fig.show(config=plotly_config)

In [24]:
np.mean([0.3149689279176914, 0.31449457458602254, 0.31066798538596907, 0.3219519143488398])

0.3155208505596307

<h1 align='center'>Thanks and make sure to learn!</h1>